In [27]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import io
import os
import sys
import time
import json
from IPython.display import display
from time import strftime, gmtime
import sagemaker
from sagemaker.predictor import csv_serializer
# import libraries
import boto3, re, sys, math, json, os, sagemaker, urllib.request, random
from sagemaker import get_execution_role
import numpy as np                                
import pandas as pd                               
import matplotlib.pyplot as plt                   
from IPython.display import Image                 
from IPython.display import display               
from time import gmtime, strftime                 
from sagemaker.predictor import csv_serializer   
from pandas.api.types import CategoricalDtype
from sklearn.model_selection import train_test_split
import xgboost as xgb


In [28]:
#split the downloaded data into train/test/validation files
FILE_TRAIN = 'train'
FILE_VALIDATION = 'validation'
FILE_TEST = 'test'
columns = ['WILL_ATTACH_IN_60', 'ORGANIZATION_ID', 
       'PLAN_TIER', 'BILLING_PLAN_ID', 'CC_FEE', 'ASP', 'INDUSTRY', 'ORGSIZE',
       'BANK_CONNECTED', 'BANK_VERIFIED', 'CREATED', 'SCHEDULED', 'OMW',
       'STARTED', 'IN_PROGRESS', 'ESTIMATED', 'INVOICED', 'TOTALPAYMENT_COUNT',
       'TOTALPAYMENT_AMOUNT', 'ONLINE_BOOKINGS', 'CC_COUNT', 'CC_AMOUNT',
       'INSTAPAY_AMOUNT', 'ACH_COUNT', 'CASH_COUNT', 'CASH_AMOUNT',
       'CHECK_COUNT', 'CHECK_AMOUNT', 'OTHER_COUNT', 'DAYS_SINCE_ENROLLMENT',
       'REASON_TO_ADOPT', 'STAGENAME_PRODUCT', 'CC_COUNT_ALL',
       'AVG_TIME_TO_PAYMENT', 'COMMITMENTS_ALL', 'COMMITMENTS', 'CC_PERC',
       'CASH_PERC', 'CHECK_PERC', 'OTHER_PERC', 'ACH_PERC']
columns_new = ['label', 'ORGANIZATION_ID', 
       'PLAN_TIER', 'BILLING_PLAN_ID', 'CC_FEE', 'ASP', 'INDUSTRY', 'ORGSIZE',
       'BANK_CONNECTED', 'BANK_VERIFIED', 'CREATED', 'SCHEDULED', 'OMW',
       'STARTED', 'IN_PROGRESS', 'ESTIMATED', 'INVOICED', 'TOTALPAYMENT_COUNT',
       'TOTALPAYMENT_AMOUNT', 'ONLINE_BOOKINGS', 'CC_AMOUNT',
       'INSTAPAY_AMOUNT', 'ACH_COUNT', 'CASH_COUNT', 'CASH_AMOUNT',
       'CHECK_COUNT', 'CHECK_AMOUNT', 'OTHER_COUNT', 'DAYS_SINCE_ENROLLMENT',
       'REASON_TO_ADOPT', 'STAGENAME_PRODUCT', 'CC_COUNT_ALL',
       'AVG_TIME_TO_PAYMENT', 'COMMITMENTS_ALL', 'COMMITMENTS', 'CC_PERC',
       'CASH_PERC', 'CHECK_PERC', 'OTHER_PERC', 'ACH_PERC']

In [54]:
# df = pd.read_csv('../cc_attach_project.csv',index_col=0)
# print(df.shape)
# groups = df.groupby('ORGANIZATION_ID')

# split_perc = int(np.rint(df.shape[0] * 0.80))
# train_groups = set()
# test_groups = set()
# train_rows = 0
# for name, group in groups:
#     train_rows += group.shape[0]
#     train_groups.add(name)
#     if train_rows > split_perc:
#         break

(1205612, 43)


In [ ]:
# train = groups.filter(lambda x: x.name in train_groups)
# test = groups.filter(lambda x: x.name not in train_groups)
# s1 = set(train['ORGANIZATION_ID'].unique())
# s2 = set(test['ORGANIZATION_ID'].unique())
# print(len(s1.intersection(s2)))
# print(train_rows, split_perc)
# print(train.shape, test.shape)


In [29]:
def train_test_group_by_id(df, perc):
    groups = df.groupby('ORGANIZATION_ID')

    split_perc = int(np.rint(df.shape[0] * perc))
    train_groups = set()
    test_groups = set()
    train_rows = 0
    for name, group in groups:
        train_rows += group.shape[0]
        train_groups.add(name)
        if train_rows > split_perc:
            break
    train = groups.filter(lambda x: x.name in train_groups)
    test = groups.filter(lambda x: x.name not in train_groups)
    s1 = set(train['ORGANIZATION_ID'].unique())
    s2 = set(test['ORGANIZATION_ID'].unique())
    print(len(s1.intersection(s2)))
    print(train_rows, split_perc)
    print(train.shape, test.shape)
    return train, test

def assign_weight(row):
    attach_label = row['WILL_ATTACH_IN_60']
    cc_count = row['CC_COUNT']
    if attach_label == 1 and cc_count > 0:
        return 0
    elif attach_label == 0 and cc_count <= 0:
        return 1
    elif attach_label == 0 and cc_count > 0:
        return 2
    elif attach_label == 1 and cc_count <= 0:
        return 3

def get_dataset_new():
    df = pd.read_csv('../cc_attach_project.csv',index_col=0)
    print('Success: Data loaded into dataframe.')
    print(df.shape)
    # df['ENROLLMENT_DATE'] = pd.to_datetime(df['ENROLLMENT_DATE'])
    # df['DATE'] = pd.to_datetime(df['DATE'])
    df['STAGENAME_PRODUCT'] = df['STAGENAME_PRODUCT'].fillna('not called')
    df['INDUSTRY'] = df['INDUSTRY'].fillna('missing')
    df['CASH_PERC'] = df['CASH_PERC'].fillna(0.0)  
    df['CHECK_PERC'] = df['CHECK_PERC'].fillna(0.0)
    df['OTHER_PERC'] = df['OTHER_PERC'].fillna(0.0)
    df['ACH_PERC'] = df['ACH_PERC'].fillna(0.0)
    df['CC_PERC'] = df['CC_PERC'].fillna(0.0)
    # df['BILLING_PLAN_ID'] = df['BILLING_PLAN_ID'].astype('int64')
    cat_dtype = CategoricalDtype(categories=(df['PLAN_TIER'].unique()), ordered=True)
    df['PLAN_TIER'] = df['PLAN_TIER'].astype(cat_dtype) #small > medium?
    df['INDUSTRY'] = df['INDUSTRY'].astype('category')
    df['ORGSIZE'] = df['ORGSIZE'].abs()
    # df['ORGSIZE'].replace(0,1, inplace=True) #should I do this?

    #convert to all floats
    # df = df.apply(pd.to_numeric, errors='ignore', downcast='float')

    #org_id same means same org!!!
    weight = df.apply(assign_weight, axis=1)

    df.insert(0, 'label', weight, allow_duplicates=False) 
    to_drop = ['ENROLLMENT_DATE', 'DATE', 'CC_COUNT', 'WILL_ATTACH_IN_60']
    df.drop(to_drop, inplace=True, axis=1)

    df = df.reindex(columns_new, axis="columns")
#     df = df.sample(frac=1, replace=False, random_state=1) #Shuffle the df

    df = pd.get_dummies(df)
    
    print(df.shape)
#     train, test = train_test_split(df, test_size=0.15, random_state=123, shuffle=False)
    
#     print(train.shape, test.shape)
    
    return df

def get_dataset():
    df = pd.read_csv('../cc_attach_project.csv',index_col=0)
    print('Success: Data loaded into dataframe.')
    print(df.shape)

    to_drop = ['ENROLLMENT_DATE', 'DATE']
    df.drop(to_drop, inplace=True, axis=1)
    # df['ENROLLMENT_DATE'] = pd.to_datetime(df['ENROLLMENT_DATE'])
    # df['DATE'] = pd.to_datetime(df['DATE'])
    df['STAGENAME_PRODUCT'] = df['STAGENAME_PRODUCT'].fillna('not called')
    df['INDUSTRY'] = df['INDUSTRY'].fillna('missing')
    df['CASH_PERC'] = df['CASH_PERC'].fillna(0.0)  
    df['CHECK_PERC'] = df['CHECK_PERC'].fillna(0.0)
    df['OTHER_PERC'] = df['OTHER_PERC'].fillna(0.0)
    df['ACH_PERC'] = df['ACH_PERC'].fillna(0.0)
    df['CC_PERC'] = df['CC_PERC'].fillna(0.0)
    # df['BILLING_PLAN_ID'] = df['BILLING_PLAN_ID'].astype('int64')
    cat_dtype = CategoricalDtype(categories=(df['PLAN_TIER'].unique()), ordered=True)
    df['PLAN_TIER'] = df['PLAN_TIER'].astype(cat_dtype) #small > medium?
    df['INDUSTRY'] = df['INDUSTRY'].astype('category')
    df['ORGSIZE'] = df['ORGSIZE'].abs()
    # df['ORGSIZE'].replace(0,1, inplace=True) #should I do this?

    df = df.reindex(columns, axis="columns")
#     df = df.sample(frac=1, replace=False, random_state=1) #Shuffle the df

    df = pd.get_dummies(df)
    
    print(df.shape)
    train, test = train_test_split(df, test_size=0.15, random_state=123)
    
#     print(train.shape, test.shape)
    
    return (train, test)


In [30]:
train,test = get_dataset()
train.to_csv('{}.csv'.format(FILE_TRAIN), header=True, index=False)
test.to_csv('{}.csv'.format(FILE_TEST), header=True, index=False)

# train_new = get_dataset_new()
# train_new.to_csv('{}-new.csv'.format(FILE_TRAIN), header=True, index=False)
# test_new.to_csv('{}-new.csv'.format(FILE_TEST), header=True, index=False)

Success: Data loaded into dataframe.
(1205612, 43)
(1205612, 171)


In [31]:
# y_train = train['WILL_ATTACH_IN_60']
# y_test = test['WILL_ATTACH_IN_60']
# to_drop = ['WILL_ATTACH_IN_60']
# train.drop(to_drop, axis=1, inplace=True)
# test.drop(to_drop, axis=1, inplace=True)

# train = pd.get_dummies(train)
# test = pd.get_dummies(test)

# X_train, X_test, y_train, y_test = train_test_split(X, y, 
#                                                     test_size=0.2, 
#                                                     random_state=123)
# print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)
# y_train= y_train.values.reshape(-1, 1)
# y_test= y_test.values.reshape(-1, 1)

# scaler_X = preprocessing.StandardScaler().fit(X_train)
# scaler_y = preprocessing.StandardScaler().fit(y_train)
# X_train_scaled = scaler_X.transform(X_train)
# y_train_scaled = scaler_y.transform(y_train)



In [32]:
# dtrain = xgb.DMatrix(train, label=y_train)
# dtest = xgb.DMatrix(test, label=y_test)

In [33]:
# param = {'max_depth': 7, 'eta': 0.3, 'silent': 0, 'objective': 'binary:logistic'}
# param['eval_metric'] = 'logloss'
# evallist = [(dtest, 'eval'), (dtrain, 'train')]


In [34]:
# num_round = 1000
# bst = xgb.train(param, dtrain, num_round, evallist, early_stopping_rounds=10)


In [35]:


# bst.get_fscore()


In [36]:
# y_val = val['WILL_ATTACH_IN_60']
# to_drop = ['WILL_ATTACH_IN_60']
# val.drop(to_drop, axis=1, inplace=True)
# val = pd.get_dummies(val)

# dval = xgb.DMatrix(val)

# ypred = bst.predict(dval, ntree_limit=bst.best_ntree_limit)


In [37]:
# valpred = np.exp(ypred)
# from sklearn.metrics import mean_squared_error
# mean_squared_error(y_val, valpred)

In [38]:
# bst.save_model('basic-xgb.model')
# bst.dump_model('basic-xgb.raw.txt')
# text_file = open("f_importance.txt", "w")
# print("feature_importance", file=text_file)
# print(bst.get_fscore(), file=text_file)
# text_file.close()
